# Neural Nets [Network Def., Loss Fn., Backprop., Optimizers]

Neural networks can be constructed using the `torch.nn` package.

Now that you had a glimpse of autograd, nn depends on autograd to define models and differentiate them. 
An `nn.Module` contains layers, and a method `forward(input)` that returns the `output`.

For example, look at this network that classifies digit images:

<img src="img/mnist.png" />

[MNIST Example]


It is a simple feed-forward network. It takes the input, feeds it through several layers one after the other, and then finally gives the output.

A **typical training procedure** for a neural network is as follows:

1. Define the neural network that has some learnable parameters (or weights)
2. Iterate over a dataset of inputs
3. Process input through the network
4. Compute the loss (how far is the output from being correct)
5. Propagate gradients back into the network’s parameters
6. Update the weights of the network, typically using a simple update rule: 
   `weight = weight - learning_rate * gradient`


In [2]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

### Define the model architecture

In [69]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b (?)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10) # i.e. 10 classes/digits
        
    def forward(self, x: Variable) -> Variable:
        # the input & output of the forward-fn is an autograd.Variable
        # max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # if the size is a square you can also only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) # no act. func. for the last one to always generate an output
        return x
    
    def num_flat_features(self, x: Variable) -> int:
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [70]:
# the learnable parameters of a model are returned by net.parameters()

params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [71]:
input = Variable(torch.randn(1, 1, 32, 32))
out = net(input)
print(out)

# zero the gradient buffers of all params and backprops with rndm gradients:
net.zero_grad()
out.backward(torch.randn(1, 10))

Variable containing:
-0.0577  0.1013 -0.1042 -0.0948 -0.0703  0.0943  0.0724 -0.1412 -0.0528 -0.0032
[torch.FloatTensor of size 1x10]



##### (i) Note:

`torch.nn` only supports mini-batches. 
The entire `torch.nn` package only supports inputs that are a mini-batch of samples, and not a single sample.
For example, `nn.Conv2d` will take in a 4D Tensor of `nSamples x nChannels x Height x Width`.


### Recap

- `torch.Tensor`: a multi-dim. array
- `autograd.Variable`: wraps a Tensor, records the history of ops applied to it. Same API as a Tensor with some additions like `.backward()`. Holds also gradient w.r.t. the tensor.
- `nn.Module`: convient way of encapsulating params, with helpers for moving them to GPU, exporting, loading, etc.
- `nn.Parameter`: a kind of `Variable`, that is automatically registered as a parameter when assigned as an attribute to a `Module`.
- `autograd.Function`: impl. forward and backward definitions of an autograd operation. Every `Variable` op creates at least a single `Function` node, that connects to functions that created a `Variable`and encodes its history.


** ⬆️ Until now:**
- defining a neural net
- processing inputs and calling backward

** ⬇️ Now: **
- computing the loss
- updating the weights of the network


### Loss Functions

- takes the (output, target)-pair of inputs
- computes a value to estimate how far away the calculated output it from the actual target 
- there are several different loss functions under the nn package
- `nn.MSELoss` is an example loss function which compute tne mean-squared error between input + target
- See: http://pytorch.org/docs/stable/nn.html#loss-functions for more

In [72]:
output = net(input)
target = Variable(torch.arange(1, 11)) # dummy target
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

Variable containing:
 38.8000
[torch.FloatTensor of size 1]



Our computation graph would now look like this:

```
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss
```

### Backprop

- to backpropagate the error all we have to do is: `loss.backward()`
- need to **clean the existing gradients first**, otherwise they will be accumulated to existing gradients

In [74]:
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

conv1.bias.grad after backward
Variable containing:
-0.0084
-0.0171
-0.0383
 0.0250
 0.1337
 0.0375
[torch.FloatTensor of size 6]



**Read later**:
neural net package contains various modules and loss functions that form the building blocks of deep neural nets, full doc of the package is here: http://pytorch.org/docs/stable/nn.html


Now - Last thing left, for a complete model:

### Update the weights:

- **optimizers == update rules for the model**
- simplest update rule used in practice is `Stochastic Gradient Descent` (**SGD**): `weight = weight - learning_rate * gradient`
- other examples: Nesterov-SGD, Adam, RMSProp, ...
- see `torch.optim` package to see them all: http://pytorch.org/docs/stable/optim.html

In [76]:
import torch.optim as optim

# create an optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# training loop:
optimizer.zero_grad() # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # do the actual update
